# Big Data Modelling and Management 2022

Group number: 

Students (name : student number):


1.


2.

3.

4.

5.


## 🚚 BDMM Second Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the second assignment is modelling. We will use the World Wide Importers database and convert it to a document-based database. To that end, we will be leveraging concepts like data denormalization, indices, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logistics). Welcome!   <br>
Even though business is thriving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements are starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data are spread accross different systems, but we've already managed to pull them all into a mongo dump file. This data file is an exact dump of the SQL data so includes all the same structure, the SQL tables become collections and the rows become documents. This means all the original SQL keys are included in the data.<br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>

Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](./WWI.png)

**Note** You can open the file WWI.png that is in the same directory as this notebook to see the above image in more detail and zoom in as you need.

Addtionally, the WWI team asked you the deliver the following outputs in **4 weeks**:
- Understand and model the database in MongoDB.
- Setup the database so that it is performs well for the queries they have provided. You should include reasoning in comments for the decisions you make on modelling the database.
- Answer the questions (queries) on the data provided.  
- Submit the results by following the instructions.  

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

Note that WWI has the following query requirements for the database.

1. The web team needs to know:  
    1. Which state province do we have the most suppliers in?  
    2. How many people have three or more `OtherLanguage`? 
    3. Top 10 most common `OtherLanguage` for people records. 
    4. How many customer records are valid after `November 2015`? 
    5. What percentage of people records don't have the UserPreferences field? 

2. The warehouse group needs to know:  
    1. What is the average difference in days between OrderDate and ExpectedDeliveryDate for orders sold by (`SalespersonPersonID`) person with the name `Jack Potter`?
    2. Which items get ordered the most in bulk (largest average quantity ordered)?  
    3. Which two items get ordered together the most?
    4. For each customer category which 3 items have the ordered the most?
    5. What is the current stock of each stockgroup?

3. The CFO needs to know:  
    1. What is the monthly total order count for each month?  
    2. How many orders are there from the customer `Tailspin Toys (Head Office)`?
    3. What are the average monthly sales prices of all goods sold? 
    4. In each state province what is the average customer credit limit?   
    5. What are the yearly expenditures with each supplier (per supplier name)?  

4. Partnerships needs to know:  
    1. What is the most common payment type?  
    2. What percentage of people have their `Title` as `Team Member`?
    3. Which supplier of the category `Novelty Goods Supplier` has the most transactions?  
    4. What is the highest `CommissionRate` that a person has?

5. The marketing team needs to know:  
    1. What is the name of the sales person with the largest sum of invoice values in 2013 (person whose customers paid the most money)?
    2. Who are the most common `PickedByPersonID` person names for orders done by customer `Adriana Pena`?
    3. How many people have in their name the string `Sara`?
    5. What are the top 10 most Common Names (Primary or Surnames) of people?

Transform the mongo dump file provided with this notebook and model a database following mongodb's best practices. You should adjust the data model to best fit the use cases provided above. Think about collections, embedding, linking, indexing, and the patterns learned in class. Provide justifications for each decision you make. What, if any, are the trade-off's or disadvantages of your approach.

Use MongoDB queries to answer the questions on your transformed database.

### Deliverables

1. Notebook with all DB creation operations and CRUD operations to create the data model. **Important** you should include in comments justification for your decisions on modelling the data.;
2. Second notebook with all required queries and answers for the questions, **Important** please indicate with comments the steps in the data model you took to optimise each query;


# Additional Information

## Groups  

Students should form groups of at least 4 and at most 5. <br>

## Submission  Deadline

The submission includes two notebooks with outputs (cells must be run). 
Please make sure to indicate:
1. group number,
2. group members with student names and numbers,
3. the name of the database that you created. <br>

Upload the notebook on moodle before **23:59 on June 22nd**

## Evaluation   

The second homework assignment counts 40% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>

Each group submission will be evaluated on three components:
1. correctness of results;
2. simplicity and performance characteristics of the solution;
3. justification of decisions.

50% -  Database design  
50% -  Query results including performance

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups with high similarity levels in their code will undergo investigation.


In [1]:
#import libraries
import numpy as np
import pandas as pd
from pprint import pprint
import datetime

# Python connector

from pymongo import MongoClient

user="AzureDiamond"
password="hunter2"
host="localhost"
port="27017"
protocol="mongodb"

client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")
db = client.WideWorldImporters

In [ ]:
db = client.WideWorldImporters 
print(f"Database info: {db}\n")

db.name 

In [ ]:
collection_list = db.list_collection_names()

print(f"The database contains {len(collection_list)} collections")

print(f"All collections: {collection_list[0:]}")

print(f"Collection {collection_list[0]} contains {db[collection_list[0]].count_documents({})} documents")

### Customers

- The customer table is linked to a number of collections/tables in a Many-to-One relationships (Many customers can have a specific attribute, eg: Many Customers can have the same CustomerCategory). The goal here is to __reduce__ the number of linking collections/tables by __denormalizing__ the relation model and therefore __optimizing__ query time (less joins when performing the query). 
- To accomplish this, __Extended Reference Pattern__ was used, where only the necessary data (eg: CategoryName and/or ID) was merged into the new Customer collection/table.  
- cust_info_08 is the __final__ customer table

    - Merging CustomerCategories with Customers (One category has many customers)
    - Drop CustomerCategories

In [215]:
query_1 = {
        "$lookup":{
           "from": "customercategories",
           "localField": "CustomerCategoryID",
           "foreignField": "CustomerCategoryID",
           "as": "categories"
        }
    }

query_2 = {
    "$unwind": "$categories"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "DeliveryMethodID":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "PostalCityID":1,
        "DeliveryCityID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
         'ValidFrom': 1,
         'ValidTo': 1,

        "CustCatName":"$categories.CustomerCategoryName",
        "CustCatID":"$categories.CustomerCategoryID"
    }
}

query_4 = { 
        "$merge" : {
            "into" : "cust_info_01",
            "whenMatched": "replace"
        } 
    }

pipeline = [query_1,query_2,query_3,query_4]

r = db.customers.aggregate(pipeline)


In [216]:
len(list(db.cust_info_01.find()))

663

    - Merging Cities with Customers - for delivery information (One city has many customers) 

In [217]:
query_1 = {
        "$lookup":{
           "from": "cities",
           "localField": "DeliveryCityID",
           "foreignField": "CityID",
           "as": "city"
        }
    }

query_2 = {
    "$unwind": "$city"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "DeliveryMethodID":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "PostalCityID":1,
        "DeliveryCityID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
         'ValidFrom': 1,
         'ValidTo': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":"$city.CityID",
        "DeliveryCityName":"$city.CityName",
        "DeliveryCityStateID":"$city.StateProvinceID"

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_02",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_01.aggregate(pipeline)


In [218]:
len(list(db.cust_info_02.find()))

663

    - Merging Cities with Customers - for postal information (One city has many customers)

In [219]:
query_1 = {
        "$lookup":{
           "from": "cities",
           "localField": "PostalCityID",
           "foreignField": "CityID",
           "as": "city"
        }
    }

query_2 = {
    "$unwind": "$city"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "DeliveryMethodID":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "PostalCityID":1,
        "DeliveryCityID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
         'ValidFrom': 1,
         'ValidTo': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryCityStateID":1,
        "PostalCityID":"$city.CityID",
        "PostalCityName":"$city.CityName",
        "PostalCityStateID":"$city.StateProvinceID"
        

    }
}

query_4 = { 
        "$merge" : {
            "into" : "cust_info_03",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_02.aggregate(pipeline)


In [220]:
len(list(db.cust_info_03.find()))

663

    - Merging DeliveryMethod with customers (One delivery method has many customers) 

In [221]:
query_1 = {
        "$lookup":{
           "from": "deliverymethods",
           "localField": "DeliveryMethodID",
           "foreignField": "DeliveryMethodID",
           "as": "delivery"
        }
    }

query_2 = {
    "$unwind": "$delivery"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
         'ValidFrom': 1,
         'ValidTo': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryCityStateID":1,
        "PostalCityID":1,
        "PostalCityName":1,
        "PostalCityStateID":1,
        "DeliveryMethodID":"$delivery.DeliveryMethodID",
        "DeliveryMethodName":"$delivery.DeliveryMethodName"
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_04",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_03.aggregate(pipeline)


In [222]:
len(list(db.cust_info_04.find()))

663

    - Merging People with Customer - PrimaryContact (One person can have multiple customers as primary contact)

In [223]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "PrimaryContactPersonID",
           "foreignField": "PersonID",
           "as": "people"
        }
    }

query_2 = {
    "$unwind": "$people"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryCityStateID":1,
        "PostalCityID":1,
        "PostalCityName":1,
        "PostalCityStateID":1,
        "DeliveryMethodID":1,
        "DeliveryMethodName":1,
        "PrimaryContactPersonID":"$people.PersonID",
        "PrimaryContactPersonName":"$people.FullName"
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_05",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_04.aggregate(pipeline)


In [224]:
len(list(db.cust_info_05.find()))

663

    - Merging People with Customer - AlternateContact (One person can have multiple customers as alternative contact)

In [225]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "AlternateContactPersonID",
           "foreignField": "PersonID",
           "as": "people"
        }
    }

# query_2 = {
#     "$unwind": "$people"
# }


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryCityStateID":1,
        "PostalCityID":1,
        "PostalCityName":1,
        "PostalCityStateID":1,
        "DeliveryMethodID":1,
        "DeliveryMethodName":1,
        "PrimaryContactPersonID":1,
        "PrimaryContactPersonName":1,
        "AlternateContactPersonID": "$people.PersonID",
        "AlternateContactPersonName": "$people.FullName"
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_06",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_3,query_4]

r = db.cust_info_05.aggregate(pipeline)


In [226]:
len(list(db.cust_info_06.find()))

663

    - Merging Customers with State - DeliveryState (One state has many customers)

In [227]:
query_1 = {
        "$lookup":{
           "from": "stateprovinces",
           "localField": "DeliveryCityStateID",
           "foreignField": "StateProvinceID",
           "as": "state"
        }
    }

query_2 = {
    "$unwind": "$state"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryCityStateID":1,
        "DeliveryStateID":"$state.StateProvinceID",
        "DeliveryStateName": "$state.StateProvinceName",
        "PostalCityID":1,
        "PostalCityName":1,
        "PostalCityStateID":1,
        "DeliveryMethodID":1,
        "DeliveryMethodName":1,
        "PrimaryContactPersonID":1,
        "PrimaryContactPersonName":1,
        "AlternateContactPersonID": 1,
        "AlternateContactPersonName": 1,
        
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_07",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_06.aggregate(pipeline)


In [228]:
len(list(db.cust_info_07.find()))

663

    - Merging Customers with state - Postal (One state has many customers)

In [229]:
query_1 = {
        "$lookup":{
           "from": "stateprovinces",
           "localField": "PostalCityStateID",
           "foreignField": "StateProvinceID",
           "as": "state"
        }
    }

query_2 = {
    "$unwind": "$state"
}


query_3 = {
    "$project": {
        "CustomerName":1,
        "CustomerID": 1,
        "BillToCustomerID":1,
        "PhoneNumber":1,
        "BuyingGroupID":1,
        "PrimaryContactPersonID":1,
        "AlternateContactPersonID":1,
        "CreditLimit":1,
        "AccountOpenedDate":1,
        "StandardDiscountPercentage":1,
        'IsStatementSent': 1,
         'IsOnCreditHold': 1,
         'PaymentDays': 1,
         'PhoneNumber': 1,
         'FaxNumber': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'WebsiteURL': 1,
         'DeliveryAddressLine1': 1,
         'DeliveryAddressLine2': 1,
         'DeliveryPostalCode': 1,
         'DeliveryLocation': 1,
         'PostalAddressLine1': 1,
         'PostalAddressLine2':1,
         'PostalPostalCode': 1,
         'LastEditedBy': 1,
        "CustCatID":1,
        "CustCatName":1,
        "DeliveryCityID":1,
        "DeliveryCityName":1,
        "DeliveryStateID":"$state.StateProvinceID",
        "DeliveryStateName": "$state.StateProvinceName",
        
        "PostalStateID":"$state.StateProvinceID",
        "PostalStateName": "$state.StateProvinceName",
        
        "PostalCityID":1,
        "PostalCityName":1,
        "DeliveryMethodID":1,
        "DeliveryMethodName":1,
        "PrimaryContactPersonID":1,
        "PrimaryContactPersonName":1,
        "AlternateContactPersonID": 1,
        "AlternateContactPersonName": 1,
        
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "cust_info_08",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.cust_info_07.aggregate(pipeline)


In [230]:
len(list(db.cust_info_08.find()))

663

In [231]:
db.cust_info_08.find_one()

{'_id': ObjectId('6287c598636e5a12693dc817'),
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'BillToCustomerID': 1,
 'BuyingGroupID': 1,
 'CreditLimit': None,
 'AccountOpenedDate': datetime.datetime(2013, 1, 1, 0, 0),
 'StandardDiscountPercentage': Decimal128('0.000'),
 'IsStatementSent': False,
 'IsOnCreditHold': False,
 'PaymentDays': 7,
 'PhoneNumber': '(308) 555-0100',
 'FaxNumber': '(308) 555-0101',
 'DeliveryRun': '',
 'RunPosition': '',
 'WebsiteURL': 'http://www.tailspintoys.com',
 'DeliveryAddressLine1': 'Shop 38',
 'DeliveryAddressLine2': '1877 Mittal Road',
 'DeliveryPostalCode': '90410',
 'DeliveryLocation': None,
 'PostalAddressLine1': 'PO Box 8975',
 'PostalAddressLine2': 'Ribeiroville',
 'PostalPostalCode': '90410',
 'LastEditedBy': 1,
 'CustCatName': 'Novelty Shop',
 'CustCatID': 3,
 'DeliveryCityID': 19586,
 'DeliveryCityName': 'Lisco',
 'PostalCityID': 19586,
 'PostalCityName': 'Lisco',
 'DeliveryMethodID': 3,
 'DeliveryMethodName': 'Delivery Van',

### Sales Orders

- SalesOrders registers the orders performed by each customer, the goal here is also to reduce the number o joins needed to perform queries by merging relevant information of referenced tables into the Orders table.  
- Orders_04 is the __final__ orders table

    - Merging Orders with customers (One customers, many orders)

In [232]:
query_1 = {
        "$lookup":{
           "from": "cust_info_08",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "customers"
        }
    }

query_2 = {
    "$unwind": "$customers"
}


query_3 = {
    "$project": {
        '_id': 1,
         'OrderID': 1,
         'CustomerID': 1,
         'SalespersonPersonID': 1,
         'PickedByPersonID': 1,
         'ContactPersonID': 1,
         'BackorderOrderID': 1,
         'OrderDate': 1,
         'ExpectedDeliveryDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsUndersupplyBackordered': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'PickingCompletedWhen': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':'$customers.CustomerID',
        'CustomerName':'$customers.CustomerName',
        'CustomerCategory':'$customers.CustCatName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "orders_01",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.orders.aggregate(pipeline)


In [233]:
len(list(db.orders_01.find()))

73595

    - Merging Orders with Person - PickedByPersonID (The same person can pick multiple orders)

In [234]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "PickedByPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'OrderID': 1,
         'CustomerID': 1,
         'SalespersonPersonID': 1,
         'PickedByPersonID': 1,
         'ContactPersonID': 1,
         'BackorderOrderID': 1,
         'OrderDate': 1,
         'ExpectedDeliveryDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsUndersupplyBackordered': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'PickingCompletedWhen': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1, 
        "PickedByPersonID":'$ppl.PersonID',
        "PickedByName":'$ppl.FullName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "orders_02",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_3,query_4]

r = db.orders_01.aggregate(pipeline)


In [235]:
len(list(db.orders_02.find()))

73595

In [156]:
db.orders_02.find_one()

{'_id': ObjectId('6287c5ec636e5a126944ceb1'),
 'OrderID': 4,
 'SalespersonPersonID': 16,
 'ContactPersonID': 1113,
 'BackorderOrderID': None,
 'OrderDate': datetime.datetime(2013, 1, 1, 0, 0),
 'ExpectedDeliveryDate': datetime.datetime(2013, 1, 2, 0, 0),
 'CustomerPurchaseOrderNumber': '17129',
 'IsUndersupplyBackordered': True,
 'Comments': None,
 'DeliveryInstructions': None,
 'InternalComments': None,
 'PickingCompletedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'LastEditedBy': 3,
 'LastEditedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'CustomerID': 57,
 'CustomerName': 'Tailspin Toys (Lakemore, OH)',
 'CustomerCategory': 'Novelty Shop',
 'PickedByPersonID': [3],
 'PickedByName': ['Hudson Onslow']}

    - Merging Orders with Person - SalespersonPersonID (One person can perform multiple sales)

In [236]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "SalespersonPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'OrderID': 1,
         'CustomerID': 1,
         'SalespersonPersonID': 1,
         'PickedByPersonID': 1,
         'ContactPersonID': 1,
         'BackorderOrderID': 1,
         'OrderDate': 1,
         'ExpectedDeliveryDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsUndersupplyBackordered': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'PickingCompletedWhen': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1, 
        "PickedByPersonID":1,
        "PickedByName":1,
        "SalespersonPersonID":'$ppl.PersonID',
        "SalespersonPersonName":'$ppl.FullName'

    }
}
query_4 = { 
        "$merge" : {
            "into" : "orders_03",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.orders_02.aggregate(pipeline)


In [237]:
len(list(db.orders_03.find()))

73595

In [160]:
db.orders_03.find_one()

{'_id': ObjectId('6287c5ec636e5a126944ceb1'),
 'OrderID': 4,
 'ContactPersonID': 1113,
 'BackorderOrderID': None,
 'OrderDate': datetime.datetime(2013, 1, 1, 0, 0),
 'ExpectedDeliveryDate': datetime.datetime(2013, 1, 2, 0, 0),
 'CustomerPurchaseOrderNumber': '17129',
 'IsUndersupplyBackordered': True,
 'Comments': None,
 'DeliveryInstructions': None,
 'InternalComments': None,
 'PickingCompletedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'LastEditedBy': 3,
 'LastEditedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'CustomerID': 57,
 'CustomerName': 'Tailspin Toys (Lakemore, OH)',
 'CustomerCategory': 'Novelty Shop',
 'PickedByPersonID': [3],
 'PickedByName': ['Hudson Onslow'],
 'SalespersonPersonID': 16,
 'SalespersonPersonName': 'Archer Lamble'}

    - Merging Orders with Person - ContactPersonID (One person can be a contactPerson in many Orders)

In [238]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "ContactPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'OrderID': 1,
         'CustomerID': 1,
         'SalespersonPersonID': 1,
         'PickedByPersonID': 1,
         'ContactPersonID': 1,
         'BackorderOrderID': 1,
         'OrderDate': 1,
         'ExpectedDeliveryDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsUndersupplyBackordered': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'PickingCompletedWhen': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1, 
        "PickedByPersonID":1,
        "PickedByName":1,
        "SalespersonPersonID":1,
        "SalespersonPersonName":1,
        "ContactPersonID":'$ppl.PersonID',
        "ContactPersonName":'$ppl.FullName'

    }
}
query_4 = { 
        "$merge" : {
            "into" : "orders_04",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.orders_03.aggregate(pipeline)


In [239]:
len(list(db.orders_04.find()))

73595

In [2]:
db.orders_04.find_one()

{'_id': ObjectId('6287c5ec636e5a126944ceb1'),
 'OrderID': 4,
 'BackorderOrderID': None,
 'OrderDate': datetime.datetime(2013, 1, 1, 0, 0),
 'ExpectedDeliveryDate': datetime.datetime(2013, 1, 2, 0, 0),
 'CustomerPurchaseOrderNumber': '17129',
 'IsUndersupplyBackordered': True,
 'Comments': None,
 'DeliveryInstructions': None,
 'InternalComments': None,
 'PickingCompletedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'LastEditedBy': 3,
 'LastEditedWhen': datetime.datetime(2013, 1, 1, 11, 0),
 'CustomerID': 57,
 'CustomerName': 'Tailspin Toys (Lakemore, OH)',
 'CustomerCategory': 'Novelty Shop',
 'PickedByPersonID': [3],
 'PickedByName': ['Hudson Onslow'],
 'SalespersonPersonID': 16,
 'SalespersonPersonName': 'Archer Lamble',
 'ContactPersonID': 1113,
 'ContactPersonName': 'Aile Mae'}

### Sales Invoices

 - SalesInvoices is directly linked to the Sales Orders collection since one SalesOrder may have multiple SalesInvoices  
 - invoice_07 is the final SalesInvoice collection

    - Merge Sales invoices with customer ID - CustomerID (One Customer may have multiple Invoice Sales)

In [240]:
query_1 = {
        "$lookup":{
           "from": "cust_info_08",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "customers"
        }
    }

query_2 = {
    "$unwind": "$customers"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':'$customers.CustomerID',
        'CustomerName':'$customers.CustomerName',
        'CustomerCategory':'$customers.CustCatName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_01",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoices.aggregate(pipeline)


In [241]:
len(list(db.invoice_01.find()))

70510

In [5]:
db.invoice_01.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'BillToCustomerID': 832,
 'OrderID': 1,
 'DeliveryMethodID': 3,
 'ContactPersonID': 3032,
 'AccountsPersonID': 3032,
 'SalespersonPersonID': 2,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),


    - Merge Sales invoices with customer ID (Bill CustID)

In [242]:
query_1 = {
        "$lookup":{
           "from": "cust_info_08",
           "localField": "BillToCustomerID",
           "foreignField": "CustomerID",
           "as": "customers"
        }
    }

query_2 = {
    "$unwind": "$customers"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':'$customers.CustomerID',
        'BillToCustomerName':'$customers.CustomerName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_02",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_01.aggregate(pipeline)


In [243]:
len(list(db.invoice_02.find()))

70510

In [12]:
db.invoice_02.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'DeliveryMethodID': 3,
 'ContactPersonID': 3032,
 'AccountsPersonID': 3032,
 'SalespersonPersonID': 2,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'A

    - Merging Invoices with Delivery Methods

In [244]:
query_1 = {
        "$lookup":{
           "from": "deliverymethods",
           "localField": "DeliveryMethodID",
           "foreignField": "DeliveryMethodID",
           "as": "dm"
        }
    }

query_2 = {
    "$unwind": "$dm"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':1,
        'BillToCustomerName':1,
        'DeliveryMethodID':'$dm.DeliveryMethodID',
        'DeliveryMethodName':'$dm.DeliveryMethodName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_03",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_02.aggregate(pipeline)


In [245]:
len(list(db.invoice_03.find()))

70510

In [19]:
db.invoice_03.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'ContactPersonID': 3032,
 'AccountsPersonID': 3032,
 'SalespersonPersonID': 2,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'LastE

    - Merging invoices with people (ContactPersonID)

In [246]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "ContactPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':1,
        'BillToCustomerName':1,
        'DeliveryMethodID':1,
        'DeliveryMethodName':1,
        'ContactPersonID':'$ppl.PersonID',
        'ContactPersonName':'$ppl.FullName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_04",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_03.aggregate(pipeline)


In [247]:
len(list(db.invoice_04.find()))

70510

In [4]:
db.invoice_04.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'AccountsPersonID': 3032,
 'SalespersonPersonID': 2,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'LastEditedBy': 15,
 'LastEdited

    - Merging invoices with people (AccountsPersonID)

In [248]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "AccountsPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':1,
        'BillToCustomerName':1,
        'DeliveryMethodID':1,
        'DeliveryMethodName':1,
        'ContactPersonID':1,
        'ContactPersonName':1,
        'AccountPersonID':'$ppl.PersonID',
        'AccountPersonName':'$ppl.FullName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_05",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_04.aggregate(pipeline)


In [249]:
len(list(db.invoice_05.find()))

70510

In [9]:
db.invoice_05.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'AccountsPersonID': 3032,
 'SalespersonPersonID': 2,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'LastEditedBy': 15,
 'LastEdited

    - Merging invoices with people (SalesPersonID)

In [250]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "SalespersonPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'ContactPersonID': 1,
         'AccountsPersonID': 1,
         'SalespersonPersonID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':1,
        'BillToCustomerName':1,
        'DeliveryMethodID':1,
        'DeliveryMethodName':1,
        'ContactPersonID':1,
        'ContactPersonName':1,
        'AccountPersonID':1,
        'AccountPersonName':1,
        'SalespersonPersonID':'$ppl.PersonID',
        'SalespersonPersonName':'$ppl.FullName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_06",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_05.aggregate(pipeline)


In [251]:
len(list(db.invoice_06.find()))

70510

In [13]:
db.invoice_06.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'AccountsPersonID': 3032,
 'PackedByPersonID': 14,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'LastEditedBy': 15,
 'LastEditedWhen': datetime.datetime(20

    - Merging invoices with people (PackedByPersonID)

In [252]:
query_1 = {
        "$lookup":{
           "from": "people",
           "localField": "PackedByPersonID",
           "foreignField": "PersonID",
           "as": "ppl"
        }
    }

query_2 = {
    "$unwind": "$ppl"
}


query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceID': 1,
         'CustomerID': 1,
         'BillToCustomerID': 1,
         'OrderID': 1,
         'DeliveryMethodID': 1,
         'PackedByPersonID': 1,
         'InvoiceDate': 1,
         'CustomerPurchaseOrderNumber': 1,
         'IsCreditNote': 1,
         'CreditNoteReason': 1,
         'Comments': 1,
         'DeliveryInstructions': 1,
         'InternalComments': 1,
         'TotalDryItems': 1,
         'TotalChillerItems': 1,
         'DeliveryRun': 1,
         'RunPosition': 1,
         'ReturnedDeliveryData': 1,
         'ConfirmedDeliveryTime': 1,
         'ConfirmedReceivedBy': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'CustomerID':1,
        'CustomerName':1,
        'CustomerCategory':1,
        'BillToCustomerID':1,
        'BillToCustomerName':1,
        'DeliveryMethodID':1,
        'DeliveryMethodName':1,
        'ContactPersonID':1,
        'ContactPersonName':1,
        'AccountPersonID':1,
        'AccountPersonName':1,
        'SalespersonPersonID':1,
        'SalespersonPersonName':1,
        'PackedByPersonID':'$ppl.PersonID',
        'PackedByPersonName':'$ppl.FullName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoice_07",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.invoice_06.aggregate(pipeline)


In [253]:
len(list(db.invoice_07.find()))

70510

In [16]:
db.invoice_07.find_one()

{'_id': ObjectId('6287c761636e5a1269499598'),
 'InvoiceID': 1,
 'OrderID': 1,
 'InvoiceDate': datetime.datetime(2013, 1, 1, 0, 0),
 'CustomerPurchaseOrderNumber': '12126',
 'IsCreditNote': False,
 'CreditNoteReason': None,
 'Comments': None,
 'DeliveryInstructions': 'Suite 24, 1345 Jun Avenue',
 'InternalComments': None,
 'TotalDryItems': 1,
 'TotalChillerItems': 0,
 'DeliveryRun': '',
 'RunPosition': '',
 'ReturnedDeliveryData': '{"Events": [{ "Event":"Ready for collection","EventTime":"2013-01-01T12:00:00","ConNote":"EAN-125-1051"},{ "Event":"DeliveryAttempt","EventTime":"2013-01-02T07:05:00","ConNote":"EAN-125-1051","DriverID":15,"Latitude":41.3617214,"Longitude":-81.4695602,"Status":"Delivered"}],"DeliveredWhen":"2013-01-02T07:05:00","ReceivedBy":"Aakriti Byrraju"}',
 'ConfirmedDeliveryTime': datetime.datetime(2013, 1, 2, 7, 5),
 'ConfirmedReceivedBy': 'Aakriti Byrraju',
 'LastEditedBy': 15,
 'LastEditedWhen': datetime.datetime(2013, 1, 2, 7, 0),
 'CustomerID': 832,
 'CustomerName'

### Customer Transactions

    - Merge customer transactions with transaction type

In [254]:
query_1 = {
        "$lookup":{
           "from": "transactiontypes",
           "localField": "TransactionTypeID",
           "foreignField": "TransactionTypeID",
           "as": "transtype"
        }
    }

query_2 = {
    "$unwind": "$transtype"
}


query_3 = {
    "$project": {
        '_id': 1,
         'CustomerTransactionID': 1,
         'CustomerID': 1,
         'TransactionTypeID': 1,
         'InvoiceID': 1,
         'PaymentMethodID': 1,
         'TransactionDate': 1,
         'AmountExcludingTax':1,
         'TaxAmount': 1,
         'TransactionAmount': 1,
         'OutstandingBalance': 1,
         'FinalizationDate': 1,
         'IsFinalized': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'TransactionTypeID':'$transtype.TransactionTypeID',
        'TransactionTypeName':'$transtype.TransactionTypeName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "customertransactions_01",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.customertransactions.aggregate(pipeline)


In [255]:
len(list(db.customertransactions_01.find()))

97147

    - Merge customer transactions with payment method

In [256]:
query_1 = {
        "$lookup":{
           "from": "paymentmethods",
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
           "as": "paymethod"
        }
    }

query_3 = {
    "$project": {
        '_id': 1,
         'CustomerTransactionID': 1,
         'CustomerID': 1,
         'TransactionTypeID': 1,
         'InvoiceID': 1,
         'PaymentMethodID': 1,
         'TransactionDate': 1,
         'AmountExcludingTax':1,
         'TaxAmount': 1,
         'TransactionAmount': 1,
         'OutstandingBalance': 1,
         'FinalizationDate': 1,
         'IsFinalized': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'TransactionTypeID':1,
        'TransactionTypeName':1,
        'PaymentMethodID':'$paymethod.PaymentMethodID',
        'PaymentMethodName':'$paymethod.PaymentMethodName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "customertransactions_02",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_3,query_4]

r = db.customertransactions_01.aggregate(pipeline)


In [257]:
len(list(db.customertransactions_02.find()))

97147

    - Merge Customertransactions with customers (merge on the many side)
    By performing this type of merge we are pushing relevant customer data into the transaction collection which may be a good solution since one customer can have a huge number of transactions, making it difficult an inneficient to embed the transaction collection into the customer collection.

In [258]:
query_1 = {
        "$lookup":{
           "from": "cust_info_08",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "customer"
        }
    }

query_2 = {
    "$unwind": "$customer"
}


query_3 = {
    "$project": {
        '_id': 1,
         'CustomerTransactionID': 1,
         'CustomerID': 1,
         'TransactionTypeID': 1,
         'InvoiceID': 1,
         'PaymentMethodID': 1,
         'TransactionDate': 1,
         'AmountExcludingTax':1,
         'TaxAmount': 1,
         'TransactionAmount': 1,
         'OutstandingBalance': 1,
         'FinalizationDate': 1,
         'IsFinalized': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'TransactionTypeID':1,
        'TransactionTypeName':1,
        'PaymentMethodID':1,
        'PaymentMethodName':1,
        'CustomerName':'$customer.CustomerName',
        "CustomerCategory":'$customer.CustCatName',
        "PostalStateName":'$customer.PostalStateName',
        "PostalCityName": '$customer.PostalCityName'

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "customertransactions_03",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_2,query_3,query_4]

r = db.customertransactions_02.aggregate(pipeline)


In [259]:
len(list(db.customertransactions_03.find()))

97147

    - Merge customers with transactions (Merge on the one side)

In [260]:
query_1 = {
        "$lookup":{
           "from": "customertransactions_02",
           "localField": "CustomerID",
           "foreignField": "CustomerID",
           "as": "transactions"
        }
    }


query_4 = { 
        "$merge" : {
            "into" : "customer_transaction_embed",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_4]

r = db.cust_info_08.aggregate(pipeline)


In [261]:
len(list(db.customer_transaction_embed.find()))

663

In [9]:
db.customer_transaction_embed.find_one()

{'_id': ObjectId('6287c598636e5a12693dc817'),
 'CustomerID': 1,
 'CustomerName': 'Tailspin Toys (Head Office)',
 'BillToCustomerID': 1,
 'BuyingGroupID': 1,
 'CreditLimit': None,
 'AccountOpenedDate': datetime.datetime(2013, 1, 1, 0, 0),
 'StandardDiscountPercentage': Decimal128('0.000'),
 'IsStatementSent': False,
 'IsOnCreditHold': False,
 'PaymentDays': 7,
 'PhoneNumber': '(308) 555-0100',
 'FaxNumber': '(308) 555-0101',
 'DeliveryRun': '',
 'RunPosition': '',
 'WebsiteURL': 'http://www.tailspintoys.com',
 'DeliveryAddressLine1': 'Shop 38',
 'DeliveryAddressLine2': '1877 Mittal Road',
 'DeliveryPostalCode': '90410',
 'DeliveryLocation': None,
 'PostalAddressLine1': 'PO Box 8975',
 'PostalAddressLine2': 'Ribeiroville',
 'PostalPostalCode': '90410',
 'LastEditedBy': 1,
 'CustCatName': 'Novelty Shop',
 'CustCatID': 3,
 'DeliveryCityID': 19586,
 'DeliveryCityName': 'Lisco',
 'PostalCityID': 19586,
 'PostalCityName': 'Lisco',
 'DeliveryMethodID': 3,
 'DeliveryMethodName': 'Delivery Van',

### Invoice Lines

- Directly linked to Invoices, StockItem and PackageType collections, the goal of this collection is to register which products were bought in a specific invoice, including Quantity and Price. To substantially reduce query times on this collection, it was created an embeded Invoice collection where InvoiceLines was embeded into the invoice collection.

    - Merge Product infomation into Invoice Lines (One Product is in many invoice lines)

In [262]:
#embeding basic Product information in invoicelines (productName, ProductCategory)
query_1 = {
        "$lookup":{
           "from": "new_stocks4",
           "localField": "StockItemID",
           "foreignField": "StockItemID",
           "as": "stock_info"
        }
    }

query_3 = {
    "$project": {
        '_id': 1,
         'InvoiceLineID': 1,
         'InvoiceID': 1,
         'StockItemID': 1,
         'Description': 1,
         'PackageTypeID': 1,
         'Quantity': 1,
         'UnitPrice': 1,
         'TaxRate': 1,
         'TaxAmount': 1,
         'LineProfit': 1,
         'ExtendedPrice': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'StockItemID': '$stock_info.StockItemID',
        'StockItemName': '$stock_info.StockItemName',
        'StockItemCategoryName':'$stock_info.Stock_GroupName'
        
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoiceslines_01",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1, query_3,query_4]

r = db.invoicelines.aggregate(pipeline)


In [75]:
len(list(db.invoiceslines_01.find()))

228265

    - Embed InvoiceLine in Invoices (One Invoice has some Invoice Lines)
    
    Note: merging this 2 collections takes a huge amount of time so doing this every day to perform a single query isn't efficient, however there may be better solutions than this one

In [263]:
query_1 = {
        "$lookup":{
           "from": "invoicelines",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "invoice_info"
        }
    }

query_2 = {
    "$unwind": "$transactions"
}


query_3 = {
    "$project": {
        '_id': 1,
         'CustomerTransactionID': 1,
         'CustomerID': 1,
         'TransactionTypeID': 1,
         'InvoiceID': 1,
         'PaymentMethodID': 1,
         'TransactionDate': 1,
         'AmountExcludingTax':1,
         'TaxAmount': 1,
         'TransactionAmount': 1,
         'OutstandingBalance': 1,
         'FinalizationDate': 1,
         'IsFinalized': 1,
         'LastEditedBy': 1,
         'LastEditedWhen': 1,
        'TransactionTypeID':1,
        'TransactionTypeName':1,
        'PaymentMethodID':1,
        'PaymentMethodName':1,
        'CustomerName':'$customer.CustomerName',
        "CustomerCategory":'$customer.CustCatName',
        "PostalStateName":'$customer.PostalStateName',
        "PostalCityName": '$customer.PostalCityName'
        
        

    }
}
\
query_4 = { 
        "$merge" : {
            "into" : "invoices_embed",
            "whenMatched": "replace"
        } 
    }

limit = {"$limit":5}


pipeline = [query_1,query_4]

r = db.invoice_07.aggregate(pipeline)


In [3]:
db.invoices_embed.find_one()

ServerSelectionTimeoutError: localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it, Timeout: 30s, Topology Description: <TopologyDescription id: 62b1c50d6e120cfe13ac1a23, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] No connection could be made because the target machine actively refused it')>]>

In [264]:
len(list(db.invoice_07.find()))

70510

In [265]:
len(list(db.invoices_embed.find()))

70510

In [ ]:
query_22 = {
        "$lookup":{
           "from": "customertransactions_03",
           "localField": "InvoiceID",
           "foreignField": "InvoiceID",
           "as": "transactions"
        }
    }

query_23 = {
    "$unwind": "$transactions"
}

query_e = {
    '$project': {
        "transactions.TransactionAmount" : 1
    }
}

query_24 = { 
        "$merge" : {
            "into" : "invoices_embed"
        } 
    }

pipeline = [query_22, query_23, query_e ,query_24]

f =list(db.invoices_embed.aggregate(pipeline))

Joining the OrderDate information to the orderlines table, so to ease the access to data this query needs.

In [ ]:
query_1 = { 
    "$lookup":
     {
       "from": "orders_04" ,
       "localField": "OrderID",
       "foreignField": "OrderID",
       "as": "date_info"
     }
}

query_2 = {
    "$project": {
        '_id': 1,'OrderDate': 1
    }}

query_3 = {"$unwind" : "$date_info"}

query_4 = {"$merge":{"into":"new_orderlines"}}

pipeline=[query_1,query_2,query_3,query_4]

r = db.orderlines.aggregate(pipeline)

### Changes to stockitemsstockgroups

Joining the StockGroupName field to the stockitemsstockgroups collection since the stockgroups collection is almost empty, and it does not justifies its individual existence. This is an auxiliary step, since this field will later be embedded in the stockitems collection.

In [16]:
query1 = { 
    "$lookup":
     {
       "from": "stockgroups" ,
       "localField": "StockGroupID",
       "foreignField": "StockGroupID",
       "as": "Stock_Group"
     }
}

query2 = {"$unwind" : "$Stock_Group"}

query3={
    '$project': {"StockGroupID":0,"Stock_Group._id":0,"Stock_Group.StockGroupID":0,
                 "Stock_Group.LastEditedBy":0,"Stock_Group.ValidFrom":0,"Stock_Group.ValidTo":0}
}

query4 = {"$merge":{"into":"stockgroup_aux"}}

pipeline=[query1,query2,query3,query4]

result = list(db.stockitemsstockgroups.aggregate(pipeline))

#db.stockgroups.drop()

In [17]:
db.stockgroup_aux.find_one()

{'_id': ObjectId('6287c634636e5a126945eebe'),
 'LastEditedBy': 1,
 'LastEditedWhen': datetime.datetime(2013, 1, 1, 0, 0),
 'StockItemID': 1,
 'StockItemStockGroupID': 1,
 'Stock_Group': {'StockGroupName': 'Computing Novelties'}}

### Changes to stockitems -> new_stocks4

Joining the StockGroupName field to the stockitems collection since the stockitemsstockgroups collection is almost empty, and it does not justifies its individual existence.

In [18]:
query1 = { 
    "$lookup":
     {
       "from": "stockgroup_aux" ,
       "localField": "StockItemID",
       "foreignField": "StockItemID",
       "as": "Stock_GroupName"
     }
}

query2 = {"$unwind" : "$Stock_GroupName"}

query3={
    '$project': {"Stock_GroupName._id":0,"Stock_GroupName.StockItemID":0,"Stock_GroupName.LastEditedBy":0,"Stock_GroupName.LastEditedWhen":0,"Stock_GroupName.StockItemStockGroupID":0}
}

query4 = {"$merge":{"into":"new_stocks"}}

pipeline=[query1,query2,query3,query4]

a = list(db.stockitems.aggregate(pipeline))

#db.stockitemsstockgroups.drop()
#db.stockgroup_aux.drop()

In [19]:
db.new_stocks.find_one()

{'_id': ObjectId('6287c78f636e5a12694aa927'),
 'Barcode': None,
 'Brand': None,
 'ColorID': None,
 'CustomFields': '{ "CountryOfManufacture": "China", "Tags": ["USB Powered"] }',
 'InternalComments': None,
 'IsChillerStock': False,
 'LastEditedBy': 1,
 'LeadTimeDays': 14,
 'MarketingComments': 'Complete with 12 projectiles',
 'OuterPackageID': 7,
 'Photo': None,
 'QuantityPerOuter': 1,
 'RecommendedRetailPrice': Decimal128('37.38'),
 'SearchDetails': 'USB missile launcher (Green) Complete with 12 projectiles',
 'Size': None,
 'StockItemID': 1,
 'StockItemName': 'USB missile launcher (Green)',
 'Stock_GroupName': {'Stock_Group': {'StockGroupName': 'USB Novelties'}},
 'SupplierID': 12,
 'Tags': '["USB Powered"]',
 'TaxRate': Decimal128('15.000'),
 'TypicalWeightPerUnit': Decimal128('0.300'),
 'UnitPackageID': 7,
 'UnitPrice': Decimal128('25.00'),
 'ValidFrom': datetime.datetime(2016, 5, 31, 23, 11),
 'ValidTo': datetime.datetime(9999, 12, 31, 23, 59, 59, 999000)}

Joining the ColorName field to the stockitems collection since the colors collection is almost empty, and it does not justifies its individual existence.

In [20]:
query1 = { 
    '$lookup':
     {
       'from': 'colors' ,
       'localField': 'ColorID',
       'foreignField': 'ColorID',
       'as': "color"
     }
}

query2 = {'$unwind' : '$color'}

query3={
    '$project': {"ColorID":0,"color._id":0,"color.ColorID":0,"color.LastEditedBy":0,"color.ValidFrom":0,"color.ValidTo":0}
}

query4 = {"$merge":{"into":"new_stocks2"}}

pipeline=[query1,query2,query3,query4]

b = list(db.new_stocks.aggregate(pipeline))

#db.colors.drop

In [21]:
db.new_stocks2.find_one()

{'_id': ObjectId('6287c78f636e5a12694aa928'),
 'Barcode': None,
 'Brand': None,
 'CustomFields': '{ "CountryOfManufacture": "China", "Tags": ["USB Powered"] }',
 'InternalComments': None,
 'IsChillerStock': False,
 'LastEditedBy': 1,
 'LeadTimeDays': 14,
 'MarketingComments': 'Complete with 12 projectiles',
 'OuterPackageID': 7,
 'Photo': None,
 'QuantityPerOuter': 1,
 'RecommendedRetailPrice': Decimal128('37.38'),
 'SearchDetails': 'USB rocket launcher (Gray) Complete with 12 projectiles',
 'Size': None,
 'StockItemID': 2,
 'StockItemName': 'USB rocket launcher (Gray)',
 'Stock_GroupName': {'Stock_Group': {'StockGroupName': 'USB Novelties'}},
 'SupplierID': 12,
 'Tags': '["USB Powered"]',
 'TaxRate': Decimal128('15.000'),
 'TypicalWeightPerUnit': Decimal128('0.300'),
 'UnitPackageID': 7,
 'UnitPrice': Decimal128('25.00'),
 'ValidFrom': datetime.datetime(2016, 5, 31, 23, 11),
 'ValidTo': datetime.datetime(9999, 12, 31, 23, 59, 59, 999000),
 'color': {'ColorName': 'Steel Gray'}}

Joining the PackageTypeName field to the stockitems collection since the packagetypes collection is almost empty, and it does not justifies its individual existence.

In [22]:
query1 = { 
    "$lookup":
     {
       "from": "packagetypes" ,
       "localField": "PackageTypeID",
       "foreignField": "UnitPackageID",
       "as": "UnitPackage"
     }
}

query2 = {"$unwind" : "$UnitPackage"}

query3={
    '$project': {"UnitPackageID":0,"UnitPackage._id":0,"UnitPackage.PackageTypeID":0,"UnitPackage.LastEditedBy":0,"UnitPackage.ValidFrom":0,"UnitPackage.ValidTo":0}
}

query4 = {"$merge":{"into":"new_stocks3"}}

pipeline=[query1,query2,query3,query4]

result = list(db.new_stocks2.aggregate(pipeline))

In [23]:
db.new_stocks3.find_one()

{'_id': ObjectId('6287c78f636e5a12694aa928'),
 'Barcode': None,
 'Brand': None,
 'CustomFields': '{ "CountryOfManufacture": "China", "Tags": ["USB Powered"] }',
 'InternalComments': None,
 'IsChillerStock': False,
 'LastEditedBy': 1,
 'LeadTimeDays': 14,
 'MarketingComments': 'Complete with 12 projectiles',
 'OuterPackageID': 7,
 'Photo': None,
 'QuantityPerOuter': 1,
 'RecommendedRetailPrice': Decimal128('37.38'),
 'SearchDetails': 'USB rocket launcher (Gray) Complete with 12 projectiles',
 'Size': None,
 'StockItemID': 2,
 'StockItemName': 'USB rocket launcher (Gray)',
 'Stock_GroupName': {'Stock_Group': {'StockGroupName': 'USB Novelties'}},
 'SupplierID': 12,
 'Tags': '["USB Powered"]',
 'TaxRate': Decimal128('15.000'),
 'TypicalWeightPerUnit': Decimal128('0.300'),
 'UnitPackage': {'PackageTypeName': 'Tube'},
 'UnitPrice': Decimal128('25.00'),
 'ValidFrom': datetime.datetime(2016, 5, 31, 23, 11),
 'ValidTo': datetime.datetime(9999, 12, 31, 23, 59, 59, 999000),
 'color': {'ColorName'

Joining the PackageTypeName field to the stockitems collection since the packagetypes collection is almost empty, and it does not justifies its individual existence.

In [24]:
query1 = { 
    "$lookup":
     {
       "from": "packagetypes" ,
       "localField": "PackageTypeID",
       "foreignField": "OuterPackageID",
       "as": "OuterPackage"
     }
}

query2 = {"$unwind" : "$OuterPackage"}

query3={
    '$project': {"OuterPackageID":0,"OuterPackage._id":0,"OuterPackage.PackageTypeID":0,"OuterPackage.LastEditedBy":0,"OuterPackage.ValidFrom":0,"OuterPackage.ValidTo":0}
}


query4 = {"$merge":{"into":"new_stocks4"}}

pipeline=[query1,query2,query3,query4]

result = list(db.new_stocks3.aggregate(pipeline))


#db.packagetypes.drop

In [25]:
db.new_stocks4.find_one()

{'_id': ObjectId('6287c78f636e5a12694aa928'),
 'Barcode': None,
 'Brand': None,
 'CustomFields': '{ "CountryOfManufacture": "China", "Tags": ["USB Powered"] }',
 'InternalComments': None,
 'IsChillerStock': False,
 'LastEditedBy': 1,
 'LeadTimeDays': 14,
 'MarketingComments': 'Complete with 12 projectiles',
 'OuterPackage': {'PackageTypeName': 'Tube'},
 'Photo': None,
 'QuantityPerOuter': 1,
 'RecommendedRetailPrice': Decimal128('37.38'),
 'SearchDetails': 'USB rocket launcher (Gray) Complete with 12 projectiles',
 'Size': None,
 'StockItemID': 2,
 'StockItemName': 'USB rocket launcher (Gray)',
 'Stock_GroupName': {'Stock_Group': {'StockGroupName': 'USB Novelties'}},
 'SupplierID': 12,
 'Tags': '["USB Powered"]',
 'TaxRate': Decimal128('15.000'),
 'TypicalWeightPerUnit': Decimal128('0.300'),
 'UnitPackage': {'PackageTypeName': 'Tube'},
 'UnitPrice': Decimal128('25.00'),
 'ValidFrom': datetime.datetime(2016, 5, 31, 23, 11),
 'ValidTo': datetime.datetime(9999, 12, 31, 23, 59, 59, 999000)

### Changes to stockitemstransactions

Joining the TransactionTypeName field to the stockitemstransactions collection since the transactiontypes collection is almost empty, and it does not justifies its individual existence.

In [28]:
query1 = { 
    "$lookup":
     {
       "from": "transactiontypes" ,
       "localField": "TransactionTypeID",
       "foreignField": "TransactionTypeID",
       "as": "ttid"
     }
}

query2 = {"$unwind" : "$ttid"}

query3={
    '$project': {"TransactionTypeID":0,"ttid._id":0,"ttid.TransactionTypeID":0,"ttid.LastEditedBy":0,"ttid.ValidFrom":0,"ttid.ValidTo":0}
}

query4 = {"$merge":{"into":"new_stockitemstransactions"}}

pipeline=[query1,query2,query3,query4]

result = list(db.stockitemstransactions.aggregate(pipeline))

#db.transactiontypes.drop

In [29]:
db.new_stockitemstransactions.find_one()

{'_id': ObjectId('6287c64d636e5a126945f099'),
 'CustomerID': 28,
 'InvoiceID': 1325,
 'LastEditedBy': 4,
 'LastEditedWhen': datetime.datetime(2013, 1, 26, 12, 0),
 'PurchaseOrderID': None,
 'Quantity': Decimal128('-10.000'),
 'StockItemID': 213,
 'StockItemTransactionID': 6293,
 'SupplierID': None,
 'TransactionOccurredWhen': datetime.datetime(2013, 1, 26, 12, 0),
 'ttid': {'TransactionTypeName': 'Stock Issue'},
 'TransactionType_Name': {'TransactionTypeName': 'Stock Issue'}}

# Suppliers

Embed suppliercategories to collection suppliers because this information is frequently queried together and since the first collection only stores category_id and category_name it makes sense to just add this data to suppliers instead of referencing.

In [ ]:
query_1 = {
        "$lookup":{
           "from": "suppliercategories",
           "localField": "SupplierCategoryID",
           "foreignField": "SupplierCategoryID",
           "as": "SupplierCategory"
        }
    }

query_2 = {
    "$unwind": "$SupplierCategory"
}

query_3 = {
    '$project': {
        "ValidFrom" : 0, "ValidTo": 0, "WebsiteURL": 0, "SupplierCategory.SupplierCategoryID" : 0, "SupplierCategory.LastEditedBy" : 0, 
        "SupplierCategory.ValidFrom" : 0, "SupplierCategory.ValidTo" : 0, "SupplierCategory._id" : 0
    }
}

query_4 = { 
        "$merge" : {
            "into" : "Suppliers_embed"
        } 
    }

pipeline = [query_1, query_2,query_3, query_4]

r =list(db.suppliers.aggregate(pipeline))
db.Suppliers_embed.find_one() 

Embed deliverymethods to collection suppliers following the same logic of suppliercategories.
Since the collection only contains deliverymethod_id and name and this information is frequently needed when querying on suppliers, it makes sense to embed it instead of referecing it.

In [ ]:
query_5 = {
        "$lookup":{
           "from": "deliverymethods",
           "localField": "DeliveryMethodID",
           "foreignField": "DeliveryMethodID",
           "as": "DeliveryMethod"
        }
    }

query_6 = {
    "$unwind": "$DeliveryMethod"
}

query_7 = {
    '$project': {
        "DeliveryMethod.DeliveryMethodID" : 0, "DeliveryMethod.LastEditedBy" : 0, "DeliveryMethod.ValidFrom" : 0,
        "DeliveryMethod.ValidTo" : 0, "DeliveryMethod._id" : 0
    }
}

query_8 = { 
        "$merge" : {
            "into" : "Suppliers_embed"
        } 
    }

pipeline = [query_5, query_6, query_7, query_8]

a =list(db.suppliers.aggregate(pipeline))
db.Suppliers_embed.find_one()

Join collection cities and stateprovince for future embedding with collection suppliers. 
Country collection was excluded because upon investigation the country is mostly USA.
The output collection is saved in Location_embed.

In [ ]:
query_9 = {
        "$lookup":{
           "from": "cities",
           "localField": "StateProvinceID",
           "foreignField": "StateProvinceID",
           "as": "Location"
        }
    }

query_10 = {
    "$unwind": "$Location"
}

query_11 = {
    '$project': {
        "Location._id" : 0, "Location.StateProvinceID" : 0, "Location.LastEditedBy" : 0,"Location.ValidFrom" : 0, 
        "Location.ValidTo" : 0, "Location.Location" : 0 , "Location.LatestRecordedPopulation" : 0, "Border" : 0,
        "LatestRecordedPopulation" : 0, "ValidFrom" : 0, "ValidTo" : 0, "LastEditedBy" : 0
        
    }
}

query_12 = { 
        "$merge" : {
            "into" : "Location_embed"
        } 
    }

pipeline = [query_9, query_10, query_11, query_12]

b =list(db.stateprovinces.aggregate(pipeline))
db.Location_embed.find_one()

Embedding Location_embed to collection suppliers to add this data to the latter.
Since the suppliers' information will be commonly queried together with location it makes sense to embed and not just reference.

In [ ]:
query_13 = {
        "$lookup":{
           "from": "Location_embed",
           "localField": "CityID",
           "foreignField": "DeliveryCityID",
           "as": "Location"
        }
    }

query_14 = {
    "$unwind": "$Location"
}


query_15 = { 
        "$merge" : {
            "into" : "Suppliers_embed"
        } 
    }

pipeline = [query_13, query_14, query_15]

c =list(db.Suppliers_embed.aggregate(pipeline))
db.Suppliers_embed.find_one()

Embed paymentmethods to suppliertransactions collection because the information might be queried together.
Since paymentmethods only holds id and method name it is easier to embed it into suppliertransactions than to reference it.

In [ ]:
query_16 = {
        "$lookup":{
           "from": "paymentmethods",
           "localField": "PaymentMethodID",
           "foreignField": "PaymentMethodID",
           "as": "Payment_Method"
        }
    }

query_17 = {
    "$unwind": "$Payment_Method"
}

query_18 = {
    '$project': {
        "LastEditedBy" : 0, "LastEditedWhen" : 0, "Payment_Method._id" : 0, "Payment_Method.LastEditedBy" : 0, 
        "Payment_Method.ValidFrom" : 0, "Payment_Method.ValidTo" : 0
        
    }
}


query_19 = { 
        "$merge" : {
            "into" : "SupplierTransactions_embed"
        } 
    }

pipeline = [query_16, query_17, query_18, query_19]

e =list(db.suppliertransactions.aggregate(pipeline))
db.SupplierTransactions_embed.find_one()

Follows the same logic as paymentmethods; Embed transactiontypes into suppliertransactions since it only holds id and type name.
This facilitates queries that require information from both collections.

In [ ]:
query_20 = {
        "$lookup":{
           "from": "transactiontypes",
           "localField": "TransactionTypeID",
           "foreignField": "TransactionTypeID",
           "as": "TransactionType"
        }
    }

query_21 = {
    "$unwind": "$TransactionType"
}

query_22 = {
    '$project': {
        "TransactionType._id" : 0,  "TransactionType.TransactionTypeID" : 0,  "TransactionType.LastEditedBy" : 0,
         "TransactionType.ValidFrom" : 0,  "TransactionType.ValidTo" : 0
        
    }
}

query_23 = { 
        "$merge" : {
            "into" : "SupplierTransactions_embed"
        } 
    }

pipeline = [query_20, query_21, query_22, query_23]

f =list(db.suppliertransactions.aggregate(pipeline))
db.SupplierTransactions_embed.find_one()

Embed the supplier name from suppliers into supplierTransactions_embed for future querying

In [ ]:
query_22 = {
        "$lookup":{
           "from": "Suppliers_embed",
           "localField": "SupplierID",
           "foreignField": "SupplierID",
           "as": "SupplierInfo"
        }
    }

query_23 = {
    "$unwind": "$SupplierInfo"
}

query_e = {
    '$project': {
        "SupplierInfo.SupplierName" : 1
    }
}

query_24 = { 
        "$merge" : {
            "into" : "SupplierTransactions_embed"
        } 
    }

pipeline = [query_22, query_23, query_e ,query_24]

f =list(db.suppliertransactions.aggregate(pipeline))
db.SupplierTransactions_embed.find_one()

In [ ]:
query_1 = { 
    "$lookup":
     {
       "from": "Suppliers_embed" ,
       "localField": "SupplierID",
       "foreignField": "SupplierID",
       "as": "Supplier_categ"
     }
}

query_2 = {"$unwind" : "$Supplier_categ"}

query_3 = {"$project": { "Supplier_categ.SupplierCategory":1 }}

query_4 = {"$merge":{"into":"new_SupplierTransactions_embed"}}

pipeline=[query_1,query_2,query_3,query_4]

r = db.SupplierTransactions_embed.aggregate(pipeline)

In [6]:
db.stateprovinces.drop()
db.cities.drop()
db.countries.drop()

In [2]:
db.customers.drop()
db.customercategories.drop()
db.deliverymethods.drop()
db.paymentmethods.drop()

db.cust_info_01.drop()
db.cust_info_02.drop()
db.cust_info_03.drop()
db.cust_info_04.drop()
db.cust_info_05.drop()
db.cust_info_06.drop()
db.cust_info_07.drop()

In [3]:
db.orders.drop()
db.orders_01.drop()
db.orders_02.drop()
db.orders_03.drop()

In [4]:
db.customertransactions.drop()
db.customertransactions_01.drop()
db.customertransactions_02.drop()

In [5]:
db.suppliercategories.drop()
db.suppliers.drop()
db.suppliertransactions.drop()
db.SupplierTransactions_embed_00.drop()
db.SupplierTransactions_embed_01.drop()
db.Suppliers_embed_00.drop()
db.Suppliers_embed_01.drop()

In [6]:
db.colors.drop()
db.stockitems.drop()
db.new_stocks.drop()
db.new_stocks2.drop()
db.new_stocks3.drop()
db.new_stocks4.drop()
db.packagetypes.drop()
db.transactiontypes.drop()

db.stockgroups.drop()

db.stockitemstransactions.drop()
db.new_stockitemstransactions.drop()
db.stockitemsstockgroups.drop()


In [7]:
db.invoices.drop()
db.invoice_01.drop()
db.invoice_02.drop()
db.invoice_03.drop()
db.invoice_04.drop()
db.invoice_05.drop()
db.invoice_06.drop()
db.invoicelines.drop()